# Numerically Controlled Oscillator (NCO)
This notebook is used to develop the NCO which will later be developed in hardware

### Parameters

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

FS = 48_000
MIDI_RANGE = 128
A4 = 69
A4_FREQ = 440.0

N_PA  = 32       # phase accumulator width
N_LUT = 10       # LUT addr bits => 1024 entries
LUT_LEN = 1 << N_LUT

AUDIO_BITS = 24
AMP_BITS   = AUDIO_BITS - 1       # use full 24-bit dynamic range
FULLSCALE  = (1 << AMP_BITS) - 1

N_FRAC = 10     # interpolation frac bits

### **Calculate Word Values**
The mathematical equation that relates the output frequency ($F_{out}$​) of a Direct Digital Synthesis (DDS) synthesizer to the tuning word ($M$) is given by:

$F_{out}=\frac{M⋅F_{s}}{2^n}$

#### **Frequency Derivation**
$f(n) = 440\cdot2^{\frac{n-69}{12}}$
A4 (440Hz) is the 69th note

In [ ]:
notes = np.arange(MIDI_RANGE)
note_freqs = A4_FREQ * (2.0 ** ((notes - A4) / 12.0))

pa_words = np.round((note_freqs / FS) * (1 << N_PA)).astype(np.uint32)

def plot_midi_maps():
    plt.figure(); plt.plot(notes, note_freqs); plt.grid(True)
    plt.xlabel("MIDI note"); plt.ylabel("Hz"); plt.title("MIDI → Frequency")
    plt.show()

    plt.figure(); plt.plot(pa_words, note_freqs); plt.grid(True)
    plt.xlabel("Phase increment word"); plt.ylabel("Hz"); plt.title("Word → Frequency")
    plt.show()

plot_midi_maps()


In [ ]:
def q_signed(x, fullscale=FULLSCALE, dtype=np.int32):
    """Normalize to [-1,1], then quantize to signed integer range."""
    x = np.asarray(x, dtype=np.float64)
    m = np.max(np.abs(x)) + 1e-12
    x = x / m
    return np.round(x * fullscale).astype(dtype)

def lut_naive(wave, L=LUT_LEN):
    n = np.arange(L)
    if wave == "sine":
        x = np.sin(2*np.pi*n/L)
    elif wave == "saw":
        x = 2.0*(n/L) - 1.0
    elif wave == "square":
        x = np.where(n < L//2, 1.0, -1.0)
    elif wave == "triangle":
        phase = n / L
        x = 1.0 - 4.0*np.abs(phase - 0.5)
    else:
        raise ValueError("wave must be sine/saw/square/triangle")
    return q_signed(x)

def k_max_per_band(notes_per_band):
    bands = int(math.ceil(MIDI_RANGE / notes_per_band))
    Kmax = np.zeros(bands, dtype=np.int32)
    for b in range(bands):
        hi = min((b+1)*notes_per_band - 1, MIDI_RANGE-1)
        Kmax[b] = int((FS/2) // note_freqs[hi])
    return bands, Kmax

def additive_luts(wave, bands, Kmax, L=LUT_LEN):
    n = np.arange(L)
    theta = 2*np.pi*n/L

    out = []
    for b in range(bands):
        K = max(1, int(Kmax[b]))

        if wave == "saw":
            ks = np.arange(1, K+1)
            coeffs = 1.0 / ks

        elif wave == "square":
            ks = np.arange(1, K+1, 2)
            coeffs = 1.0 / ks

        elif wave == "triangle":
            ks = np.arange(1, K+1, 2)
            signs = (-1)**((ks-1)//2)
            coeffs = signs / (ks**2)

        else:
            raise ValueError("wave must be saw/square/triangle")

        x = np.sum(coeffs[:, None] * np.sin(ks[:, None] * theta[None, :]), axis=0)
        out.append(q_signed(x))
    return out

def plot_lut(x, title):
    plt.figure(figsize=(10,3))
    plt.plot(x)
    plt.grid(True)
    plt.title(title)
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.show()

In [ ]:
def nco_render(midi_note, lut, num_samples, pa_words=pa_words, wrap_mask=(LUT_LEN-1)):
    """Render one note using LUT + linear interpolation (FPGA-style)."""
    phase = np.uint32(0)
    M = np.uint32(pa_words[midi_note])

    shift_index = N_PA - N_LUT
    shift_frac  = N_PA - N_LUT - N_FRAC
    frac_mask   = (1 << N_FRAC) - 1

    y = np.zeros(num_samples, dtype=np.int32)

    for i in range(num_samples):
        phase = np.uint32(phase + M)

        idx  = int(phase >> shift_index)                  # 0..LUT_LEN-1
        frac = int((phase >> shift_frac) & frac_mask)     # 0..2^N_FRAC-1

        a = int(lut[idx])
        b = int(lut[(idx + 1) & wrap_mask])
        y[i] = a + (((b - a) * frac) >> N_FRAC)

    return y


In [ ]:
def estimate_freq_fft_parabolic(x, fs=FS):
    x = x.astype(np.float64)
    x -= np.mean(x)
    X = np.fft.rfft(x * np.hanning(len(x)))
    mag = np.abs(X)
    freqs = np.fft.rfftfreq(len(x), d=1/fs)

    k = int(np.argmax(mag))
    if k == 0 or k == len(mag)-1:
        return freqs[k]

    a = np.log(mag[k-1] + 1e-12)
    b = np.log(mag[k]   + 1e-12)
    c = np.log(mag[k+1] + 1e-12)
    p = 0.5 * (a - c) / (a - 2*b + c)   # bin offset
    return freqs[k] + p * (freqs[1] - freqs[0])

def validate_pitch(lut, step=5, seconds=1.0):
    N = int(FS * seconds)
    for n in range(0, MIDI_RANGE, step):
        y = nco_render(n, lut, N)
        f_est = estimate_freq_fft_parabolic(y)
        f_tgt = note_freqs[n]
        err_cents = 1200 * np.log2(f_est / f_tgt)
        print(f"note {n:3d}: target={f_tgt:9.4f} Hz, est={f_est:9.4f} Hz ({err_cents:+.3f} cents)")


In [ ]:
# Naive (one table each)
sin_lut      = lut_naive("sine")
saw_lut      = lut_naive("saw")
square_lut   = lut_naive("square")
triangle_lut = lut_naive("triangle")

plot_lut(sin_lut, "Naive sine LUT")
plot_lut(saw_lut, "Naive saw LUT")
plot_lut(square_lut, "Naive square LUT")
plot_lut(triangle_lut, "Naive triangle LUT")

# Band-limited (additive synthesis)
NOTES_PER_BAND = 6
BANDS, Kmax = k_max_per_band(NOTES_PER_BAND)

square_luts   = additive_luts("square",   BANDS, Kmax)
saw_luts      = additive_luts("saw",      BANDS, Kmax)
triangle_luts = additive_luts("triangle", BANDS, Kmax)

plot_lut(square_luts[0],   f"Square band 0 (Kmax={Kmax[0]})")
plot_lut(square_luts[-1],  f"Square band {BANDS-1} (Kmax={Kmax[-1]})")


In [ ]:
from IPython.display import Audio

def to_int16(x):
    x = np.asarray(x, dtype=np.int64)
    # scale down to int16 for playback (keeps shape, avoids clipping)
    m = np.max(np.abs(x)) + 1e-12
    y = (x / m) * 32767.0
    return np.int16(np.clip(np.round(y), -32768, 32767))

def get_band(midi_note, notes_per_band=NOTES_PER_BAND):
    return min(midi_note // notes_per_band, BANDS-1)

def play(midi_note, wave="sine", seconds=0.25):
    N = int(FS * seconds)
    if wave == "sine":
        lut = sin_lut
    elif wave == "square":
        lut = square_luts[get_band(midi_note)]
    elif wave == "saw":
        lut = saw_luts[get_band(midi_note)]
    elif wave == "triangle":
        lut = triangle_luts[get_band(midi_note)]
    else:
        raise ValueError("wave must be sine/square/saw/triangle")

    y = nco_render(midi_note, lut, N)
    return Audio(to_int16(y), rate=FS)

play(69, "square", 0.5)


In [ ]:
def sweep(wave, dur=0.08):
    return np.concatenate([nco_render(n, (
        sin_lut if wave=="sine" else
        square_luts[get_band(n)] if wave=="square" else
        saw_luts[get_band(n)] if wave=="saw" else
        triangle_luts[get_band(n)]
    ), int(FS*dur)) for n in range(MIDI_RANGE)])

Audio(to_int16(sweep("saw")), rate=FS)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

WAVES = ["square", "saw", "triangle", "sine"]

def pick_lut(midi_note, wave):
    if wave == "sine":
        return sin_lut
    b = get_band(midi_note)
    return {"square": square_luts, "saw": saw_luts, "triangle": triangle_luts}[wave][b]

def morph_render(morph, midi_note=A4, seconds=0.02):
    # morph: 0..(len(WAVES)*2^N_FRAC-1)
    max_m = len(WAVES) * (1<<N_FRAC) - 1
    morph = int(np.clip(morph, 0, max_m))

    seg = morph >> N_FRAC
    fade = morph & ((1<<N_FRAC)-1)

    w0 = WAVES[seg]
    w1 = WAVES[(seg+1) % len(WAVES)]
    lut0 = pick_lut(midi_note, w0)
    lut1 = pick_lut(midi_note, w1)

    N = int(FS * seconds)
    y0 = nco_render(midi_note, lut0, N)
    y1 = nco_render(midi_note, lut1, N)
    y  = y0 + (((y1 - y0) * fade) >> N_FRAC)
    return y, w0, w1, fade

morph_slider = widgets.IntSlider(
    value=0, min=0, max=len(WAVES)*(1<<N_FRAC)-1,
    description="morph", continuous_update=True,
    layout=widgets.Layout(width="800px"),
    style={'description_width': 'initial'},
)
out = widgets.Output()

def update(*_):
    with out:
        clear_output(wait=True)
        y, w0, w1, fade = morph_render(morph_slider.value, midi_note=69, seconds=0.02)
        plt.figure(figsize=(10,3))
        plt.plot(y); plt.grid(True)
        plt.title(f"{w0} → {w1} | fade={fade}/{(1<<N_FRAC)-1}")
        plt.show()

morph_slider.observe(update, names="value")
display(morph_slider, out)
update()
